##### Imports:

In [11]:
from utils import separate_for_training, preprocess, calculate_emission_from_chroma, calculate_mu_from_chroma, calculate_transition_probabilites, format_indiv_chroma, predict, get_unique_predicted, calculate_initial_probabilities
import pickle
from chroma import get_chromagram
import pandas as pd
from tqdm import tqdm
import numpy as np
from hmmlearn import hmm

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [12]:
# Load data and split into training and test
piece_name_dict = preprocess('dataset.pkl')

training_piece_names, _, test_piece_names = separate_for_training(piece_name_dict, 0.8, 0.)

with open(r"dataset.pkl", 'rb') as data:
    midi_data = pickle.load(data)

100%|██████████| 5762/5762 [00:05<00:00, 1046.94it/s]


##### Create Chromagram from Training Data:

In [13]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    formatted = format_indiv_chroma(indiv_chroma)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(200)

100%|██████████| 4609/4609 [01:19<00:00, 58.32it/s]


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,151,0,0,0,0,145,0,0,0,0,126,Bm
1,0,151,0,0,0,0,145,0,0,0,0,126,Bm
2,0,151,0,0,0,0,145,0,0,0,0,126,Bm
3,0,151,0,0,0,0,145,0,0,0,0,126,Bm
4,0,151,0,0,0,0,145,0,0,0,0,126,Bm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,0,0,0,75,0,0,0,0,0,0,0,0,G#m
27,0,0,0,104,0,0,89,0,270,0,0,87,G#m
28,0,0,0,71,0,0,82,0,292,0,0,82,G#m
29,0,0,0,71,0,0,0,0,0,0,0,0,G#m


##### Create HMM Components:

###### Initial State Probabilities:

In [14]:
initial_state_probabilties = calculate_initial_probabilities(training_piece_names, midi_data)
initial_state_probabilties

Niko_Kotoulas_Sus2_Bm-D-A-Bm (vi-I-V-vi).mid
Niko_Kotoulas_Back_And_Forth_Arp_3_A-Bm-G-D (V-vi-IV-I).mid
Niko_Kotoulas__RhythmChordProg_5_E-B-F#-G#m (IV-I-V-vi).mid
Niko_Kotoulas_Melody_7_G#m-F#-D#m-E (vi-V-iii-IV) - 115-130bpm.mid
Niko_Kotoulas__RhythmChordProg_2_D-Em-G (I-ii-IV).mid
Niko_Kotoulas_Drop_Pianos_4_B-C#m-E (I-ii-IV).mid
Niko_Kotoulas_Melody_4_G#m-F#-E-D#m (vi-V-IV-iii) - 115-130bpm.mid
Niko_Kotoulas_Plucks_4_B-F#-D#m-G#m (I-V-iii-vi).mid
Niko_Kotoulas_Melody_8_B-E-F#-B (I-IV-V-I) - 130-160bpm.mid
Niko_Kotoulas_Melody_1_G#m-D#m-E (vi-iii-IV) - 130-160bpm.mid
Niko_Kotoulas_Melody_9_Bm-A-Bm-G (vi-V-vi-IV) - 160-180bpm.mid
Niko_Kotoulas__RhythmChordProg_4_D-A-C# (I-V-VII).mid
Niko_Kotoulas_Drop_Pianos_2_G-Bm-F#m (IV-vi-iii).mid
Niko_Kotoulas_Plucks_5_D-A-C# (I-V-VII).mid
Niko_Kotoulas_Emotional_Chords_18_E-B-G#m-D#m_.mid
Niko_Kotoulas_ChordProg_6_Bm-A-G-F#m (vi-V-IV-iii).mid
Niko_Kotoulas_Melody_1_Em-G-Bm-A (ii-IV-vi-V) - 160-180bpm.mid
Niko_Kotoulas_ChordProg_5_D-A-C# (I-V-V

C        0.000000
Cm       0.000000
Cdim     0.000000
C#       0.000000
C#m      0.190879
C#dim    0.010206
D        0.048208
Dm       0.000000
Ddim     0.000000
D#       0.000000
D#m      0.049294
D#dim    0.000000
E        0.000434
Em       0.000000
Edim     0.000000
F        0.000217
Fm       0.000000
Fdim     0.000000
F#       0.021716
F#m      0.053855
F#dim    0.000000
G        0.094028
Gm       0.000000
Gdim     0.000000
G#       0.000000
G#m      0.042128
G#dim    0.000000
A        0.009772
Am       0.000000
Adim     0.000000
A#       0.000000
A#m      0.000000
A#dim    0.010206
B        0.189142
Bm       0.279913
Bdim     0.000000
dtype: float64

###### Transition Matrix:

In [15]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
transition_prob_matrix

C:\Users\julia\CS4100\final\CS4100_Final\utils.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[8.60640301e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.55682020e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.51653018e-04 0.00000000e+00 1.21412804e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.08775037e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.54664484e-05 0.00000000e+00 2.13797035e-04
 0.00000000e+00 0.00000000e+00 2.77777778e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.31821777e-04 0.00000000e+00]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_chords_matrix.update(transition_prob_matrix)
C:\Users\julia\CS4100\final\CS4100_Final\utils.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise i

,C,Cm,Cdim,C#,C#m,C#dim,D,Dm,Ddim,D#,...,G#dim,A,Am,Adim,A#,A#m,A#dim,B,Bm,Bdim
C,0.860640,0,0,0,0.000000,0.001883,0.001883,0,0,0.000000,...,0,0.005650,0,0,0.000000,0,0.000000,0.007533,0.003766,0
Cm,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,...,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0
Cdim,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,...,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0
C#,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,...,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0
C#m,0.000000,0,0,0,0.859168,0.000000,0.001357,0,0,0.000000,...,0,0.000452,0,0,0.000000,0,0.000000,0.008896,0.004674,0
C#dim,0.000000,0,0,0,0.007752,0.875969,0.025840,0,0,0.000000,...,0,0.002584,0,0,0.000000,0,0.000000,0.010336,0.012920,0
D,0.000356,0,0,0,0.002223,0.000000,0.862173,0,0,0.000000,...,0,0.053263,0,0,0.000089,0,0.000000,0.004179,0.027299,0
Dm,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,...,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0
Ddim,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,...,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0
D#,0.000000,0,0,0,0.000000,0.000000,0.062500,0,0,0.875000,...,0,0.000000,0,0,0.000000,0,0.000000,0.031250,0.000000,0


###### Mu Value:

In [16]:
mu = calculate_mu_from_chroma(chromagram)
mu

C     11.632498
C#    37.284792
D     25.149188
D#    33.891449
E     29.344477
F     16.440707
F#    44.816597
G     16.657540
G#    40.515652
A     22.194425
A#    24.906934
B     49.349622
dtype: float64

###### Emission Matrix:

In [17]:
emission = calculate_emission_from_chroma(chromagram)

In [18]:
emission

array([[[ 3.18985373e+02, -1.97579989e+02,  3.60435011e+01, ...,
          2.13321970e+02, -7.70184011e+01, -1.17422433e+01],
        [-1.97579989e+02,  4.53717077e+03, -3.85448733e+02, ...,
         -1.11102284e+03,  9.80249107e+02, -4.79788606e+01],
        [ 3.60435011e+01, -3.85448733e+02,  9.43730643e+02, ...,
          6.47990855e+02, -3.48972126e+02, -7.37101710e+01],
        ...,
        [ 2.13321970e+02, -1.11102284e+03,  6.47990855e+02, ...,
          7.69525400e+03, -2.53298852e+03,  4.37016438e+02],
        [-7.70184011e+01,  9.80249107e+02, -3.48972126e+02, ...,
         -2.53298852e+03,  3.39634244e+03, -3.81811962e+02],
        [-1.17422433e+01, -4.79788606e+01, -7.37101710e+01, ...,
          4.37016438e+02, -3.81811962e+02,  1.62955370e+03]],

       [[ 7.63718310e+02,  0.00000000e+00, -4.86046948e+02, ...,
         -3.40056338e+02,  2.95511737e+02, -3.00769953e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e

In [19]:
model = hmm.GaussianHMM(n_components=transition_prob_matrix.shape[0], covariance_type="Full")
print(transition_prob_matrix.shape)
print(initial_state_probabilties.shape)
model.startprob_ = initial_state_probabilties
model.transmat_ = transition_prob_matrix.values
model.means_ = mu

(36, 36)
(36,)


In [20]:
chroma_with_preds = predict(transition_prob_matrix, model, mu)

ValueError: transmat_ rows must sum to 1 (got row sums of [1.         0.         0.         0.         0.99984922 1.
 1.         0.         0.         1.         0.99966182 0.
 1.         1.         0.         1.         0.         0.
 1.         1.         0.         1.         0.         0.
 1.         1.         0.         0.99992873 0.         0.
 1.         0.         1.         0.99990869 1.         0.        ])